In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import pandas as pd
import re as re
import matplotlib.pyplot as plt
import xgboost as xgb

In [3]:
def myPrint(x):
    print(x.head(15))
    print(x.info())
def myPrint2(x):
    print(x)

In [4]:
test_a_wa_df = pd.read_csv('../data/Test-A/test_a_wa_df.csv', header = 0)

In [5]:
myPrint(test_a_wa_df)

     uid    wa_name  visit_cnt  visit_dura  up_flow  down_flow  wa_type  date
0   5000   360安全中心网          2        1408      530        515        0     5
1   5000    DNS查询流量         16        2753     2856       2432        0    29
2   5000    DNS查询流量         87       29164    20315      16538        0    30
3   5000    DNS查询流量        143       87255    28821      24335        0    31
4   5000  GROWINGIO          1        1074      676        340        0     3
5   5000       IZAT          1        1711     2069      32133        0    14
6   5000       IZAT          2        2454     1831      32919        0    16
7   5000       IZAT          2          52     2118      33234        0     9
8   5000       IZAT          2          59     1802      32883        0    26
9   5000       IZAT          4       10241     4016      66196        0     6
10  5000       IZAT          4        8537     3728      66014        0    19
11  5000       IZAT          5        1900     3725      66135  

# Feature 1

In [8]:
tmp = test_a_wa_df.uid.value_counts()
tmp = tmp.sort_index()

In [11]:
myPrint2(tmp)

5000     612
5001    1337
5002     135
5003    1988
5004     854
5005      22
5006    4127
5007    1473
5008     232
5009     367
5010     416
5011    2186
5012     624
5013     405
5014     652
5015      76
5016     695
5017    1056
5018      59
5019      64
5020    1529
5021     287
5022     770
5023     116
5024     285
5025       2
5026    1039
5027     796
5028      65
5029     765
        ... 
6969    2875
6970     540
6971     268
6972       5
6973    2907
6974     382
6975     605
6976    1223
6977    4193
6978    1768
6979    3306
6980     902
6981    2226
6982    1650
6983     956
6984     211
6985     476
6986    2524
6987      26
6988    1774
6989    1361
6990    2577
6992     577
6993    1253
6994     561
6995     444
6996     828
6997    2563
6998     286
6999    2120
Name: uid, Length: 1977, dtype: int64


In [14]:
dict = tmp.to_dict()
tmp2 = pd.Series(index = range(5000, 7000), data = dict)

In [15]:
tmp2 = tmp2.fillna(0)
tmp2 = tmp2.astype('int64')

In [16]:
myPrint2(tmp2)

5000     612
5001    1337
5002     135
5003    1988
5004     854
5005      22
5006    4127
5007    1473
5008     232
5009     367
5010     416
5011    2186
5012     624
5013     405
5014     652
5015      76
5016     695
5017    1056
5018      59
5019      64
5020    1529
5021     287
5022     770
5023     116
5024     285
5025       2
5026    1039
5027     796
5028      65
5029     765
        ... 
6970     540
6971     268
6972       5
6973    2907
6974     382
6975     605
6976    1223
6977    4193
6978    1768
6979    3306
6980     902
6981    2226
6982    1650
6983     956
6984     211
6985     476
6986    2524
6987      26
6988    1774
6989    1361
6990    2577
6991       0
6992     577
6993    1253
6994     561
6995     444
6996     828
6997    2563
6998     286
6999    2120
Length: 2000, dtype: int64


In [17]:
tmp2.to_csv('../data/Test-A/test_a_feature_1.csv', index=True)

# Feature 2 to 5

In [18]:
tmp = test_a_wa_df[['uid', 'visit_cnt', 'visit_dura', 'up_flow', 'down_flow']].groupby('uid').sum()

In [19]:
dict = tmp.to_dict()
tmp2 = pd.DataFrame(index = range(5000, 7000), data = dict)

In [20]:
tmp2 = tmp2.fillna(0)
tmp2.loc[:, ['down_flow', 'up_flow', 'visit_dura', 'visit_cnt']] = tmp2.loc[:, ['down_flow', 'up_flow', 'visit_dura', 'visit_cnt']].astype('int64')

In [22]:
tmp2.to_csv('../data/Test-A/test_a_feature_2to5.csv', index=True)

# Feature 6

In [23]:
group = test_a_wa_df.groupby('uid')['wa_name']

In [24]:
agg = group.aggregate({'wa_name': lambda x: x.nunique()})

d:\program files (x86)\python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [25]:
agg.rename(columns=lambda x:x.replace('wa_name','wa_cnt'), inplace=True)

In [26]:
dict = agg.to_dict()
agg2 = pd.DataFrame(index = range(5000, 7000), data = dict)

In [27]:
agg2 = agg2.fillna(0)
agg2 = agg2.astype('int64')

In [28]:
agg2.to_csv('../data/Test-A/test_a_feature_6.csv', index=True)

# Feature 21

In [6]:
group = test_a_wa_df.groupby('uid')['visit_cnt']
print(group.unique())

uid
5000    [2, 16, 87, 143, 1, 4, 5, 6, 8, 9, 12, 3, 11, ...
5001    [1, 4, 5, 13, 15, 17, 55, 97, 12, 31, 2, 3, 6,...
5002    [222, 5, 3, 1, 108, 51, 2, 70, 13, 68, 33, 34,...
5003    [3, 4, 7, 13, 21, 73, 108, 200, 6, 8, 10, 11, ...
5004    [1, 6, 7, 10, 5, 9, 15, 3, 14, 18, 19, 56, 27,...
5005                           [4, 6, 27, 73, 1, 2, 5, 3]
5006    [1, 2, 4, 5, 21, 3, 6, 8, 9, 12, 11, 14, 26, 4...
5007    [1, 5, 2, 4, 3, 6, 153, 184, 275, 7, 9, 14, 12...
5008    [2, 9, 8, 195, 1, 3, 7, 28, 208, 5, 13, 192, 2...
5009    [27, 18, 1, 13, 11, 36, 2, 24, 43, 7, 6, 9, 35...
5010    [2, 5, 3, 194, 718, 1, 4, 10, 18, 6, 67, 240, ...
5011    [2, 5, 1, 4, 6, 8, 9, 11, 12, 13, 15, 16, 17, ...
5012    [5, 56, 14, 1, 2, 3, 7, 88, 158, 4, 6, 10, 59,...
5013    [3, 2, 1, 6, 5, 11, 4, 44, 9, 15, 7, 19, 26, 3...
5014    [2, 1, 23, 24, 4, 16, 7, 3, 5, 6, 8, 10, 11, 1...
5015    [7, 82, 2, 4, 13, 16, 3, 5, 1, 8, 15, 6, 19, 3...
5016    [1, 5, 2, 23, 58, 200, 3, 4, 6, 7, 8, 9, 11, 1...
5017    [1

In [7]:
visit_cnt = group.agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_')

In [8]:
print(visit_cnt)

      wa_visit_cnt_std  wa_visit_cnt_max  wa_visit_cnt_min  \
uid                                                          
5000        170.879780              1622                 1   
5001         57.959731               782                 1   
5002         28.143628               222                 1   
5003        237.786261              2627                 1   
5004        302.626132              4266                 1   
5005         15.549182                73                 1   
5006        265.600954              7352                 1   
5007         84.349941               904                 1   
5008         30.084431               233                 1   
5009        129.321297              1503                 1   
5010         93.740281               761                 1   
5011         97.519171              1253                 1   
5012         46.495561               352                 1   
5013         19.810819               265                 1   
5014    

# Feature 22

In [9]:
visit_dura = test_a_wa_df.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_')

In [10]:
print(visit_dura)

      wa_visit_dura_std  wa_visit_dura_max  wa_visit_dura_min  \
uid                                                             
5000      177260.254870            1591028                  0   
5001       80780.174514            1269158                  0   
5002       49838.122387             396336                  0   
5003      260554.382817            2811463                  0   
5004      395520.920389            6042713                  0   
5005       25806.195954             124279                  0   
5006      346204.645519           10031334                  0   
5007      122254.853875            1458010                  0   
5008       49961.544724             422978                  0   
5009      186611.777121            2316625                  0   
5010      114328.086400            1025650                  0   
5011      123042.550907            1660360                  0   
5012       47865.919713             350862                  0   
5013       22382.533478  

# Feature 23

In [11]:
up_flow = test_a_wa_df.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_')

In [12]:
print(up_flow)

      wa_up_flow_std  wa_up_flow_max  wa_up_flow_min  wa_up_flow_median  \
uid                                                                       
5000    1.562213e+06        15470330               0             9767.0   
5001    4.293216e+05        12490922               0             4290.0   
5002    1.468730e+05         1134938               0             6427.0   
5003    4.014819e+05        10170905               0            13765.5   
5004    7.515589e+05        11604645               0             5936.5   
5005    3.531728e+04          141354              65             3684.0   
5006    3.887818e+06       157947685               0             3512.0   
5007    6.160469e+05        15166091               0             7534.0   
5008    9.857105e+04          872200               0             2672.5   
5009    9.345490e+05        15162723               0             6083.0   
5010    5.495695e+05         6501426               0            11866.0   
5011    2.874133e+05     

# Feature 24

In [13]:
down_flow = test_a_wa_df.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_')

# Feature 25

NameError: name 'test_b_wa_df' is not defined

# Merge Feature 21 to 24

In [14]:
agg = visit_cnt
agg = agg.join(visit_dura)
agg = agg.join(up_flow)
agg = agg.join(down_flow)

In [15]:
print(agg)

      wa_visit_cnt_std  wa_visit_cnt_max  wa_visit_cnt_min  \
uid                                                          
5000        170.879780              1622                 1   
5001         57.959731               782                 1   
5002         28.143628               222                 1   
5003        237.786261              2627                 1   
5004        302.626132              4266                 1   
5005         15.549182                73                 1   
5006        265.600954              7352                 1   
5007         84.349941               904                 1   
5008         30.084431               233                 1   
5009        129.321297              1503                 1   
5010         93.740281               761                 1   
5011         97.519171              1253                 1   
5012         46.495561               352                 1   
5013         19.810819               265                 1   
5014    

In [16]:
dict = agg.to_dict()
agg2 = pd.DataFrame(index = range(5000, 7000), data = dict)

In [17]:
agg2 = agg2.fillna(0)
agg2 = agg2.astype('int64')

In [18]:
agg2.to_csv('../data/Test-A/test_a_feature_21to24.csv', index=True, header=None)